In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from matplotlib.lines import Line2D

'''
The code below works with a velocity vector field across discrete time steps

The code will model the evoluation of a particle dropped randomly into the vector field

Code utilizes a utilities file "Field_utils" that contains helper functions

Your data should be parsed into two arrays:  1) x-direction velocities for each grid point across time steps; 2) y-direction velocities for
each grid point across time steps

e.g. a 504 x 505 x 100 array of velocities in the x-direction (horizontal direction)


'''

#call in the x and y directional fields from your data source
x_veloc,y_veloc = .....

#vector field shape
# e.g. a 504 x 555 x 100 grid would lead to x_shape = x_veloc.shape[1] = 555; y_shape = x_veloc.shape[0] = 504; time_shape = x_veloc.shape[2]


time_delta = ...  #units corresponding to the time differential between successive vector fields
num_steps = ...  #number of directional velocity steps taken within each time interval
h = time_delta/num_steps

simulations = ...


    
for j in range(simulations): 
    position_history_x = []  #set up a list to keep track of the positions across time
    position_history_y = []  
    random_start = np.array([np.random.rand()*y_shape, np.random.rand()*x_shape]) #shape (2,)
    current_position = random_start
    position_history_x.append(current_position[0])  
    position_history_y.append(current_position[1])
    
    for t in range(time_shape):
        n = 1   
        while n*h <=time_delta:
            x,y = current_position[0],current_position[1]
            active_x,active_y = Field_utils.active_grid(x,y)
            new_x = max(min(x_veloc[active_x,active_y,t]*h + x,(y_shape -1),0)
            new_y = max(min(y_veloc[active_x,active_y,t]*h + y,(x_shape -1),0)
            current_position = np.array([new_x,new_y])
            position_history_x.append(current_position[0])
            position_history_y.append(current_position[1])
            n = n+1
            
    plt.plot(position_history_x,position_history_y,markevery=[0],marker="x",markersize=6)
   
    
plt.axis([0,y_shape,0,x_shape])
plt.gca().set_aspect(1)
plt.title("Simulations of Flows for a uniformly \nrandomly placed particle/object")
plt.show()


In [ ]:
'''
Next, we can  parametrize a Gaussian process to model the time dynamics of the velocity vector field.  This
will ultimately allow one to model varying time intervals for the vector field
Step (1):  Assume a Gaussian process for the x and y directional vectors over time
Step (2):  Assume a Radial Basis Function Kernel (RBF) for the Covariance dynamics
Step (3):  Use cross-validation to choose the RBF paremeters for your model

The outputs of this analysis will be conditional means and conditional covariance matrices that can be used
for inference over different time intervals

Below, I show how this can be done for a single grid point
'''

#pick a location of your liking from your vector field
#example:
loc = [95, 335]

u = np.zeros((time_shape,2))  #first column is index, second column is vector
v = np.zeros((time_shape,2))  

#initiate two arrays holding time index and velocity vector
for i in range(time_shape):
    u[i,0] = i
    u[i,1] = x_veloc[loc[0],loc[1],i]
    v[i,0] = i
    v[i,1] = y_veloc[loc[0],loc[1],i]


#Set up initial parameters for RBF and cross validation:
#example:
sigma = .05  #for radial basis kernel
L = 15  #for radiol basis kernel
r = .001
k = 10  #k-fold cross validation...will give time_shape/10 = 10 data points for testing

#will need means for the stochastic x and y vector variables...here will use the average across the time_shape data points
mean_u = np.mean(u[:,1])
mean_v = np.mean(v[:,1])

#choose your hyperparameters for cross-validation
#example:
sigmas = np.linspace(.03,.1,5)
Ls = np.linspace(90,150,20)

##set up a table for results
results = np.zeros((Ls.size,sigmas.size))

#first do for the u-vectors
for l in Ls:
    for s in sigmas:
        cum_likelihood = 1

        #first set up calcs for the u vectors 
        for i in range(k):
            beg = int(100/k*i)
            end = int(100/k*(i+1))
            valid = np.vstack((u[0:beg,:],u[end:,:]))  
            test = u[beg:end,:]
            #now I have the validation and test partitions, with the first columns being the indices/distance metric
            cov11 = Field_utils.kernel_1(test,s,l)
            cov22 = Field_utils.kernel_1(valid,s,l)
            cov12 = Field_utils.kernel_2(test,valid,s,l)
            #now carry out the conditional mean and variance calculations:
            cond_mean = Field_utils.cond_mean(mean_u,mean_u,cov12,cov22,r,valid[:,1].reshape(-1,1))
            cond_var = Field_utils.cond_var(cov11,cov12,cov22,r)
            #next, compute the likelihood of the test data on the conditional distribution
            likelihood = multivariate_normal.pdf(test[:,1].reshape(-1,),mean=cond_mean.reshape(-1,),cov=cond_var)
            cum_likelihood = cum_likelihood*likelihood
                 
        results[np.argwhere(Ls==l),np.argwhere(sigmas==s)] = round(np.log(cum_likelihood),2)

#now for the v-vectors

sigmas_v = np.linspace(.03,.1,5)
Ls_v = np.linspace(90,150,20)

##set up a table for results
results_v = np.zeros((Ls_v.size,sigmas_v.size))

for l in Ls_v:
    for s in sigmas_v:
        cum_likelihood = 1

        #first set up calcs for the v vectors 
        for i in range(k):
            beg = int(100/k*i)
            end = int(100/k*(i+1))
            valid = np.vstack((v[0:beg,:],v[end:,:]))  
            test = v[beg:end,:]
            #now I have the validation and test partitions, with the first columns being the indices/distance metric
            cov11 = Field_utils.kernel_1(test,s,l)
            cov22 = Field_utils.kernel_1(valid,s,l)
            cov12 = Field_utils.kernel_2(test,valid,s,l)
            #now carry out the conditional mean and variance calculations:
            cond_mean = Field_utils.cond_mean(mean_v,mean_v,cov12,cov22,r,valid[:,1].reshape(-1,1))
            cond_var = Field_utils.cond_var(cov11,cov12,cov22,r)
            #next, compute the likelihood of the test data on the conditional distribution
            likelihood = multivariate_normal.pdf(test[:,1].reshape(-1,),mean=cond_mean.reshape(-1,),cov=cond_var)
            cum_likelihood = cum_likelihood*likelihood
            
        
        results_v[np.argwhere(Ls_v==l),np.argwhere(sigmas_v==s)] = round(np.log(cum_likelihood),2)


results_pd = pd.DataFrame(results,index=np.round(Ls,0),columns=sigmas)
results_pd.index.name = 'L value'
results_pd.columns.name = 'Sigma'


results_pd.style.background_gradient(axis= None, cmap='Greens',vmin=80.0)


results_pd_v = pd.DataFrame(results_v,index=np.round(Ls_v,0),columns=sigmas_v)
results_pd_v.index.name = 'L value'
results_pd_v.columns.name = 'Sigma'

results_pd_v.style.background_gradient(axis= None, cmap='Greens',vmin=70.0)


In [ ]:
'''
You can also utilize SKLearn's GaussianProcessorRegressor functionality
to determine the appropriate kernel parameters.  Use the below code and compare the
Kernel output parameters to those you derived via cross-validation

'''
import sklearn as sk
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

#instantiate the regressor objects
GPR_u = 1**2*GaussianProcessRegressor()  #placing the 1**2 in front allows the GP regressor to tune for sigma
GPR_v = 1**2*GaussianProcessRegressor()

#fit to your data
GPR_u.fit(u[:,0].reshape(-1,1),u[:,1].reshape(-1,1))
GPR_v.fit(v[:,0].reshape(-1,1),v[:,1].reshape(-1,1))

GPR_u.kernel_
GPR_v.kernel_



In [ ]:
'''
Next, we can utilize our GP modelling to interpolate the time dynamics of the vector field

Step (1):  Determine what time intervals you want to analyze; e.g. if your velocity field data has 100 time stamps where
measurement intervals represent 1 hour, then you can fine tune your model to interpolate the dynamics of the velocity field for 
10 minute intervals for example.

Step (2):  Use your chosen kernel hyperparameters from the above analysis

'''

#The code below assumes your original vector field data was measured 100 different times, i.e. 100 time stamps
#Your goal was to interpolate the vector field to obtain a total of 300 time stamps:  the 100 original plus 200 interpolated
#you can change these  below by adjusting the code 

u_L_opt = ...
u_sigma_opt = ...
v_L_opt = ...
v_sigma_opt = ...


unobs_vector = np.zeros((200))
for i in range(100):
    unobs_vector[i*2] = i + 1.3333
    unobs_vector[(i*2+1)] = i + 1.6666


obs_vector = np.linspace(1,100,100)

size_unobs = unobs_vector.shape[0]
size_obs = obs_vector.shape[0]
cov_unobs_u = np.zeros((size_unobs,size_unobs))
cov_unobs_v = np.zeros((size_unobs,size_unobs))
cov_obs_u = np.zeros((size_obs,size_obs))
cov_obs_v = np.zeros((size_obs,size_obs))

for i in range(size_unobs):
        for j in range(size_unobs):
            cov_unobs_u[i,j] = Field_utils.kernel_func(unobs_vector[i],unobs_vector[j],u_sigma_opt,u_L_opt)

for i in range(size_unobs):
        for j in range(size_unobs):
            cov_unobs_v[i,j] = Field_utils.kernel_func(unobs_vector[i],unobs_vector[j],v_sigma_opt,v_L_opt)

for i in range(size_obs):
        for j in range(size_obs):
            cov_obs_u[i,j] = Field_utils.kernel_func(obs_vector[i],obs_vector[j],u_sigma_opt,u_L_opt)

for i in range(size_obs):
        for j in range(size_obs):
            cov_obs_v[i,j] = Field_utils.kernel_func(obs_vector[i],obs_vector[j],v_sigma_opt,v_L_opt)


## now you have the square covariance matrix for my unobserved flows and my observed flows

#compute cross covariances between obs and unobs

cross_cov_u = np.zeros((size_unobs,size_obs))
cross_cov_v = np.zeros((size_unobs,size_obs))

for i in range(size_unobs):
    for j in range(size_obs):
        cross_cov_u[i,j] = Field_utils.kernel_func(unobs_vector[i],obs_vector[j],u_sigma_opt,u_L_opt)

for i in range(size_unobs):
    for j in range(size_obs):
        cross_cov_v[i,j] = Field_utils.kernel_func(unobs_vector[i],obs_vector[j],v_sigma_opt,v_L_opt)


#compute conditional means and conditional variances for the u and v-velocities
#the blue dots on the plot will be the observed data, while the red dots will be the interpolated data

u_cond_mean_unobs = HW5_utils.cond_mean(mean_u,mean_u,cross_cov_u,cov_obs_u,r,u[:,1])
print(u_cond_mean_unobs.shape)
u_cond_cov_unobs = HW5_utils.cond_var(cov_unobs_u,cross_cov_u,cov_obs_u,r)
print(u_cond_cov_unobs.shape)

v_cond_mean_unobs = HW5_utils.cond_mean(mean_v,mean_v,cross_cov_v,cov_obs_v,r,v[:,1])
print(v_cond_mean_unobs.shape)
v_cond_cov_unobs = HW5_utils.cond_var(cov_unobs_v,cross_cov_v,cov_obs_v,r)


#set up a variance vector for each unobs point
u_var_unobs_top = np.zeros((200))
u_var_unobs_bottom = np.zeros((200))
for i in range(200):
    u_var_unobs_top[i] = u_cond_mean_unobs[i,0] + np.sqrt(u_cond_cov_unobs[i,i])*3
    u_var_unobs_bottom[i] = u_cond_mean_unobs[i,0] - np.sqrt(u_cond_cov_unobs[i,i])*3
    

plt.plot(unobs_vector.reshape(-1,1),u_var_unobs_top.reshape(-1,1),color='0.8')
plt.plot(unobs_vector.reshape(-1,1),u_var_unobs_bottom.reshape(-1,1),color='0.8')
plt.fill_between(unobs_vector,u_var_unobs_top,u_var_unobs_bottom,color='0.8')
plt.scatter(unobs_vector.reshape(-1,1),u_cond_mean_unobs,label="conditional mean",color='red')
plt.scatter(obs_vector.reshape(-1,1),u[:,1],label="observed u_velocities",color='blue')
            

plt.legend()
plt.show

v_var_unobs_top = np.zeros((200))
v_var_unobs_bottom = np.zeros((200))

for i in range(200):
    v_var_unobs_top[i] = v_cond_mean_unobs[i,0] + np.sqrt(v_cond_cov_unobs[i,i])*3
    v_var_unobs_bottom[i] = v_cond_mean_unobs[i,0] - np.sqrt(v_cond_cov_unobs[i,i])*3
    


plt.plot(unobs_vector.reshape(-1,1),v_var_unobs_top.reshape(-1,1),color='0.8')
plt.plot(unobs_vector.reshape(-1,1),v_var_unobs_bottom.reshape(-1,1),color='0.8')
plt.fill_between(unobs_vector,v_var_unobs_top,v_var_unobs_bottom,color='0.8')
plt.scatter(unobs_vector.reshape(-1,1),v_cond_mean_unobs,label="conditional mean",color='red')
plt.scatter(obs_vector.reshape(-1,1),v[:,1],label="observed v_velocities",color='blue')

plt.legend()
plt.show